In [2]:
import rasterio as rio, numpy as np


In [9]:
with rio.open('./HAT/datasets/WV3/WV3_test_LR_linear/3band/050258288120_000_Patch_WV3_ps_x4.tif') as src:
    a = src.read([1,2,3]).astype(np.float32)  # 너의 3band가 실제 RGB인지 점검
    a = a.reshape(3,-1)
    print('corr R-G', np.corrcoef(a[0],a[1])[0,1],
          'G-B', np.corrcoef(a[1],a[2])[0,1],
          'R-B', np.corrcoef(a[0],a[2])[0,1])
# 모두 ~0.99 이상이면 사실상 동일 채널 -> 밴드 선택/정규화 파이프라인 점검


corr R-G 0.9870873351927273 G-B 0.9843006649505859 R-B 0.9758429894108454


In [24]:
import rasterio as rio
import numpy as np

p = "./HAT/datasets/WV3/WV3_test_LR_linear/3band_u8/050258288120_000_Patch_WV3_ps_x4.tif"
# p = "./HAT/datasets/WV3/WV3_test_HR/3band/050258288120_000_Patch_WV3_ps.tif"

with rio.open(p) as src:
    print("bands:", src.count, "size:", (src.height, src.width))
    print("descriptions:", src.descriptions)          # 밴드 설명(있을 경우)
    print("colorinterp:", src.colorinterp)            # RGB 태그 여부
    arr = src.read()                                  # (C,H,W)
    print("per-channel stats:")
    for i in range(src.count):
        a = arr[i].astype(np.float32)
        q1, q99 = np.percentile(a, [1,99])
        print(f" Band{i+1}: min={a.min():.1f}, max={a.max():.1f}, q1={q1:.1f}, q99={q99:.1f}, std={a.std():.1f}")
    # 채널 간 상관
    c = arr.reshape(arr.shape[0], -1).astype(np.float32)
    for (i,j) in [(0,1),(1,2),(0,2)]:
        r = np.corrcoef(c[i], c[j])[0,1]
        print(f" corr Band{i+1}-Band{j+1}: {r:.6f}")


bands: 3 size: (128, 128)
descriptions: ('WV3_B05 (Red)', 'WV3_B03 (Green)', 'WV3_B02 (Blue)')
colorinterp: (<ColorInterp.red: 3>, <ColorInterp.green: 4>, <ColorInterp.blue: 5>)
per-channel stats:
 Band1: min=9.0, max=255.0, q1=14.0, q99=181.0, std=31.1
 Band2: min=17.0, max=255.0, q1=25.0, q99=175.2, std=27.2
 Band3: min=14.0, max=216.0, q1=20.0, q99=161.0, std=23.6
 corr Band1-Band2: 0.987068
 corr Band2-Band3: 0.984304
 corr Band1-Band3: 0.977245


In [20]:
import rasterio as rio
import numpy as np

# p = "./HAT/datasets/WV3/WV3_test_LR_linear/3band/050258288120_000_Patch_WV3_ps_x4.tif"
p = "./HAT/datasets/WV3/WV3_train_LR_linear/3band_u16/x4/050258288070_326_Patch_WV3_ps_x4.tif"

# p = "./HAT/datasets/WV3/WV3_test_LR_linear/3band/050258288120_000_Patch_WV3_ps_x4.tif"

with rio.open(p) as src:
    print("bands:", src.count, "size:", (src.height, src.width))
    print("descriptions:", src.descriptions)          # 밴드 설명
    print("colorinterp:", src.colorinterp)            # RGB 태그 여부
    arr = src.read()                                  # (C,H,W)
    print("per-channel stats:")
    for i in range(src.count):
        a = arr[i].astype(np.float32)
        q1, q99 = np.percentile(a, [1,99])
        print(f" Band{i+1}: min={a.min():.1f}, max={a.max():.1f}, q1={q1:.1f}, q99={q99:.1f}, std={a.std():.1f}")
    # 채널 간 상관
    c = arr.reshape(arr.shape[0], -1).astype(np.float32)
    for (i,j) in [(0,1),(1,2),(0,2)]:
        r = np.corrcoef(c[i], c[j])[0,1]
        print(f" corr Band{i+1}-Band{j+1}: {r:.6f}")


bands: 3 size: (128, 128)
descriptions: ('WV3_B05 (Red)', 'WV3_B03 (Green)', 'WV3_B02 (Blue)')
colorinterp: (<ColorInterp.red: 3>, <ColorInterp.green: 4>, <ColorInterp.blue: 5>)
per-channel stats:
 Band1: min=2763.0, max=55886.0, q1=3177.8, q99=19733.8, std=3184.5
 Band2: min=2843.0, max=46040.0, q1=3234.0, q99=18328.4, std=2793.7
 Band3: min=2737.0, max=44728.0, q1=3061.5, q99=16094.9, std=2426.2
 corr Band1-Band2: 0.965869
 corr Band2-Band3: 0.984882
 corr Band1-Band3: 0.930966


In [1]:
from pathlib import Path
from collections import Counter

BASE = Path("/home/nayeon/SR/HAT/HAT/datasets/WV3")
VARIANTS = ["fc", "rgb"]              # 검사할 변형
SPLITS   = ["train", "valid", "test"] # 분할

def extract_key(p: Path) -> str:
    """파일명에서 '_Patch' 앞까지를 공통 키로 추출."""
    return p.stem.split("_Patch", 1)[0]

def list_tifs(dirpath: Path):
    files = sorted(dirpath.glob("*.tif"))
    keys  = {extract_key(p) for p in files}
    names = {p.name for p in files}
    return files, keys, names

for var in VARIANTS:
    print(f"\n=== [{var}] ===")
    total_hr = total_lr = 0
    for split in SPLITS:
        hr_dir = BASE / var / "HR" / split
        lr_dir = BASE / var / "LR" / split

        hr_files, hr_keys, hr_names = list_tifs(hr_dir)
        lr_files, lr_keys, lr_names = list_tifs(lr_dir)

        # 개수 출력
        print(f"[{var}/{split}] HR={len(hr_files):,}  LR={len(lr_files):,}")
        total_hr += len(hr_files)
        total_lr += len(lr_files)

        # 키 기준 동등성 검사 (가장 중요)
        only_hr = hr_keys - lr_keys
        only_lr = lr_keys - hr_keys
        if not only_hr and not only_lr:
            print(" - OK: HR/LR 키 집합 일치(샘플 페어링 정상)")
        else:
            print(f" - HR에만 있는 키: {len(only_hr)}개")
            print(f" - LR에만 있는 키: {len(only_lr)}개")
            for i, k in enumerate(sorted(list(only_hr))[:5], 1):
                print(f"   * only HR (예시 {i}): {k}")
            for i, k in enumerate(sorted(list(only_lr))[:5], 1):
                print(f"   * only LR (예시 {i}): {k}")

        # (옵션) 파일명까지 완전 동일한지 확인 — LR이 _x4 등 접미사 다르면 불일치가 정상
        if hr_names == lr_names:
            print(" - 참고: 파일명(확장자 포함)도 HR/LR 완전히 동일")
        else:
            print(" - 참고: 파일명은 다를 수 있음(예: LR에 '_x4'), 키 기준으로 비교 중")

        # (옵션) 중복 키 경고
        hr_dup = [k for k, c in Counter([extract_key(p) for p in hr_files]).items() if c > 1]
        lr_dup = [k for k, c in Counter([extract_key(p) for p in lr_files]).items() if c > 1]
        if hr_dup:
            print(f" - [WARN] HR/{split} 중복 키 {len(hr_dup)}개 (예: {hr_dup[:5]})")
        if lr_dup:
            print(f" - [WARN] LR/{split} 중복 키 {len(lr_dup)}개 (예: {lr_dup[:5]})")

    print(f"[{var}] 합계: HR={total_hr:,}  LR={total_lr:,}")



=== [fc] ===
[fc/train] HR=11,820  LR=11,820
 - OK: HR/LR 키 집합 일치(샘플 페어링 정상)
 - 참고: 파일명은 다를 수 있음(예: LR에 '_x4'), 키 기준으로 비교 중
[fc/valid] HR=1,477  LR=1,477
 - OK: HR/LR 키 집합 일치(샘플 페어링 정상)
 - 참고: 파일명은 다를 수 있음(예: LR에 '_x4'), 키 기준으로 비교 중
[fc/test] HR=1,479  LR=1,479
 - OK: HR/LR 키 집합 일치(샘플 페어링 정상)
 - 참고: 파일명은 다를 수 있음(예: LR에 '_x4'), 키 기준으로 비교 중
[fc] 합계: HR=14,776  LR=14,776

=== [rgb] ===
[rgb/train] HR=11,820  LR=11,820
 - OK: HR/LR 키 집합 일치(샘플 페어링 정상)
 - 참고: 파일명은 다를 수 있음(예: LR에 '_x4'), 키 기준으로 비교 중
[rgb/valid] HR=1,477  LR=1,477
 - OK: HR/LR 키 집합 일치(샘플 페어링 정상)
 - 참고: 파일명은 다를 수 있음(예: LR에 '_x4'), 키 기준으로 비교 중
[rgb/test] HR=1,479  LR=1,479
 - OK: HR/LR 키 집합 일치(샘플 페어링 정상)
 - 참고: 파일명은 다를 수 있음(예: LR에 '_x4'), 키 기준으로 비교 중
[rgb] 합계: HR=14,776  LR=14,776
